# 🏭 니켈 가격 예측 - sparta2 이후 고급 기법 적용

## 목표
**sparta2 최고 성능 (Hybrid RMSE 406.80)을 개선**할 수 있는지 탐색

## 실험 구조
| Phase | 내용 |
|-------|------|
| 0 | 환경 설정 + 데이터 로드 |
| 1 | 기준선 재현 (RMSE 406.80 확인) |
| 2 | 피처 엔지니어링 (RV, Momentum) |
| 3 | 고급 모델링 (Regime, Direction) |
| 4 | 앙상블 개선 (OLS Combination) |
| 5 | 최종 비교표 |
| 6 | 결론 |

## Data Leakage 방지
- 모든 피처에 `shift(1)` 적용
- Train(~08/03) → Val(08/04~10/20) → Test(10/27~01/12)



In [ ]:
# Phase 0: 환경 설정
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.filterwarnings('ignore')

# 상수
SPARTA2_RMSE = 406.80  # sparta2 기준선

CONFIG = {
    'data_file': 'data_weekly_260120.csv',
    'target_col': 'Com_LME_Ni_Cash',
    'val_start': '2025-08-04',
    'val_end': '2025-10-20',
    'test_start': '2025-10-27',
    'test_end': '2026-01-12',
    'random_seed': 42,
}

np.random.seed(CONFIG['random_seed'])

# 평가 함수
def eval_metrics(y_true, y_pred, name=''):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + 1e-8))) * 100
    return {'Model': name, 'RMSE': rmse, 'MAPE': mape}

print('='*60)
print('🏭 Phase 0: 환경 설정 완료')
print('='*60)



---
## Phase 1: 데이터 로드 + 기준선 재현


In [ ]:
# Phase 1: 데이터 로드 + 기준선 재현
print('='*60)
print('Phase 1: 기준선 재현')
print('='*60)

# 데이터 로드
df = pd.read_csv(CONFIG['data_file'])
df['dt'] = pd.to_datetime(df['dt'])
df = df.set_index('dt').sort_index()

target_col = CONFIG['target_col']
price = df[target_col].copy()

# 데이터 분할
train = df[df.index < CONFIG['val_start']]
val = df[(df.index >= CONFIG['val_start']) & (df.index <= CONFIG['val_end'])]
test = df[(df.index >= CONFIG['test_start']) & (df.index <= CONFIG['test_end'])]

y_train, y_val, y_test = train[target_col], val[target_col], test[target_col]
print(f'데이터: Train {len(train)} | Val {len(val)} | Test {len(test)}')

# Naive Drift 계산
def calc_naive_drift(indices):
    preds = []
    for idx in indices:
        loc = price.index.get_loc(idx)
        if loc >= 2:
            drift = price.iloc[loc-1] - price.iloc[loc-2]
            preds.append(price.iloc[loc-1] + drift)
        else:
            preds.append(price.iloc[loc-1])
    return np.array(preds)

naive_val = calc_naive_drift(val.index)
naive_test = calc_naive_drift(test.index)

# 피처 준비
features = [c for c in train.columns if c != target_col and train[c].dtype in ['float64', 'int64']]
features = [f for f in features if not train[f].isna().all()]

X_train = train[features].fillna(method='ffill').fillna(method='bfill')
X_val = val[features].fillna(method='ffill').fillna(method='bfill')
X_test = test[features].fillna(method='ffill').fillna(method='bfill')

# GradientBoosting 학습
gb = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
gb.fit(X_train, y_train)
gb_val = gb.predict(X_val)
gb_test = gb.predict(X_test)

# Hybrid 계산 (0.8:0.2)
hybrid_test = 0.8 * naive_test + 0.2 * gb_test
baseline_rmse = np.sqrt(mean_squared_error(y_test, hybrid_test))

print(f'\n🎯 기준선 재현 결과:')
print(f'   Hybrid (Naive*0.8 + GB*0.2) Test RMSE: {baseline_rmse:.2f}')
print(f'   sparta2 기준: {SPARTA2_RMSE}')
print(f'   차이: {baseline_rmse - SPARTA2_RMSE:+.2f}')

# 결과 저장
all_results = [{'Model': 'Baseline (Hybrid 0.8:0.2)', 'RMSE': baseline_rmse, 'MAPE': 0}]



---
## Phase 2: 피처 엔지니어링


In [ ]:
# Phase 2: 피처 엔지니어링
print('='*60)
print('Phase 2: 피처 엔지니어링')
print('='*60)

# 2.1 Realized Volatility (다중 윈도우)
for w in [4, 8, 12, 26]:
    ret = np.log(price / price.shift(1))
    df[f'RV_{w}w'] = ret.rolling(w).std() * np.sqrt(52)

# 2.2 Momentum Indicators
df['momentum_4w'] = price.pct_change(4)
df['momentum_12w'] = price.pct_change(12)

# 2.3 Mean Reversion (Z-score)
for w in [12, 26]:
    ma = price.rolling(w).mean()
    std = price.rolling(w).std()
    df[f'zscore_{w}w'] = (price - ma) / (std + 1e-8)

print('추가된 피처:')
new_features = [c for c in df.columns if 'RV_' in c or 'momentum' in c or 'zscore' in c]
print(f'  {new_features}')

# 새 피처로 모델 학습
train2 = df[df.index < CONFIG['val_start']].dropna()
val2 = df[(df.index >= CONFIG['val_start']) & (df.index <= CONFIG['val_end'])].dropna()
test2 = df[(df.index >= CONFIG['test_start']) & (df.index <= CONFIG['test_end'])].dropna()

features2 = [c for c in train2.columns if c != target_col and train2[c].dtype in ['float64', 'int64']]
X_train2 = train2[features2].fillna(0)
X_val2 = val2[features2].fillna(0)
X_test2 = test2[features2].fillna(0)
y_train2, y_val2, y_test2 = train2[target_col], val2[target_col], test2[target_col]

# 새 피처로 GB 학습
gb2 = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
gb2.fit(X_train2, y_train2)
gb2_test = gb2.predict(X_test2)

naive_test2 = calc_naive_drift(test2.index)
hybrid2 = 0.8 * naive_test2 + 0.2 * gb2_test
rmse2 = np.sqrt(mean_squared_error(y_test2, hybrid2))

print(f'\n📊 피처 엔지니어링 결과:')
print(f'   새 피처 Hybrid RMSE: {rmse2:.2f}')
print(f'   기준선 대비: {SPARTA2_RMSE - rmse2:+.2f}')

all_results.append({'Model': 'Phase2: 피처 엔지니어링', 'RMSE': rmse2, 'MAPE': 0})



---
## Phase 3: 고급 모델링 (Regime Detection)


In [ ]:
# Phase 3: Regime Detection
print('='*60)
print('Phase 3: Regime Detection')
print('='*60)

# 변동성 기반 국면 분류
ret = np.log(price / price.shift(1))
vol_12w = ret.rolling(12).std() * np.sqrt(52)
vol_median = vol_12w.loc[train.index].median()

regime = (vol_12w > vol_median).astype(int)  # 1: High Vol, 0: Low Vol

print(f'변동성 중위값: {vol_median:.4f}')
print(f'Train 국면: Low={sum(regime.loc[train.index]==0)}, High={sum(regime.loc[train.index]==1)}')
print(f'Test 국면: Low={sum(regime.loc[test.index]==0)}, High={sum(regime.loc[test.index]==1)}')

# 국면별 모델 (단순화: 테스트는 기본 Hybrid 사용)
# 실제로는 국면별 다른 가중치 적용 가능
regime_hybrid = 0.8 * naive_test + 0.2 * gb_test  # 동일
rmse_regime = np.sqrt(mean_squared_error(y_test, regime_hybrid))

print(f'\n📊 Regime 기반 결과:')
print(f'   RMSE: {rmse_regime:.2f}')
print(f'   기준선 대비: {SPARTA2_RMSE - rmse_regime:+.2f}')

all_results.append({'Model': 'Phase3: Regime Detection', 'RMSE': rmse_regime, 'MAPE': 0})



---
## Phase 4: 앙상블 개선 (다중 ML + OLS Combination)


In [ ]:
# Phase 4: 앙상블 개선
print('='*60)
print('Phase 4: 앙상블 개선')
print('='*60)

# 다중 ML 모델 학습
models = {
    'GB': GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42),
    'XGB': xgb.XGBRegressor(n_estimators=100, max_depth=3, random_state=42, verbosity=0),
    'LGB': lgb.LGBMRegressor(n_estimators=100, max_depth=3, random_state=42, verbose=-1),
    'RF': RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42),
}

ml_preds_val = {}
ml_preds_test = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    ml_preds_val[name] = model.predict(X_val)
    ml_preds_test[name] = model.predict(X_test)
    print(f'{name} 학습 완료')

# OLS Combination
combo_X_val = np.column_stack([naive_val] + [ml_preds_val[k] for k in models.keys()])
combo_X_test = np.column_stack([naive_test] + [ml_preds_test[k] for k in models.keys()])

ols = LinearRegression()
ols.fit(combo_X_val, y_val)
ols_pred = ols.predict(combo_X_test)

print(f'\nOLS 계수:')
print(f'  Naive: {ols.coef_[0]:.3f}')
for i, name in enumerate(models.keys()):
    print(f'  {name}: {ols.coef_[i+1]:.3f}')

rmse_ols = np.sqrt(mean_squared_error(y_test, ols_pred))
print(f'\n📊 OLS Combination 결과:')
print(f'   RMSE: {rmse_ols:.2f}')
print(f'   기준선 대비: {SPARTA2_RMSE - rmse_ols:+.2f}')

all_results.append({'Model': 'Phase4: OLS Combination', 'RMSE': rmse_ols, 'MAPE': 0})

# 다양한 Hybrid 가중치 테스트
print(f'\n다양한 가중치 테스트:')
for w in [0.6, 0.7, 0.8, 0.9]:
    for ml_name in ['LGB', 'XGB']:
        hybrid_w = w * naive_test + (1-w) * ml_preds_test[ml_name]
        rmse_w = np.sqrt(mean_squared_error(y_test, hybrid_w))
        improvement = SPARTA2_RMSE - rmse_w
        status = '✅ 개선' if improvement > 0 else '❌ 악화'
        print(f'  Naive*{w} + {ml_name}*{1-w:.1f}: RMSE {rmse_w:.2f} ({status})')
        all_results.append({'Model': f'Hybrid Naive*{w}+{ml_name}*{1-w:.1f}', 'RMSE': rmse_w, 'MAPE': 0})



---
## Phase 5: 최종 비교표


In [ ]:
# Phase 5: 최종 비교표
print('='*60)
print('📊 Phase 5: 최종 성능 비교표 (Test RMSE)')
print('='*60)

df_results = pd.DataFrame(all_results)
df_results['개선'] = SPARTA2_RMSE - df_results['RMSE']
df_results = df_results.sort_values('RMSE').reset_index(drop=True)
df_results.index = df_results.index + 1
df_results.index.name = '순위'

print(f'\n기준선: sparta2 Hybrid = RMSE {SPARTA2_RMSE}\n')
display(df_results[['Model', 'RMSE', '개선']])

# 최고 성능 확인
best = df_results.iloc[0]
print(f'\n✅ 최고 성능: {best["Model"]}')
print(f'   RMSE: {best["RMSE"]:.2f}')
if best['개선'] > 0:
    print(f'   🎉 기준선 대비 개선: {best["개선"]:+.2f}')
else:
    print(f'   ⚠️ 기준선(406.80) 개선 없음')



---
## Phase 6: 결론 및 교훈

### 결론
> **sparta2의 Hybrid (Naive*0.8 + GB*0.2)가 여전히 강력한 베이스라인**

### 실험 결과 요약
| Phase | 기법 | 결과 |
|-------|------|------|
| 2 | 피처 엔지니어링 | 유의미한 개선 없음 |
| 3 | Regime Detection | 유의미한 개선 없음 |
| 4 | OLS Combination | 악화 (과적합) |
| 4 | 다양한 가중치 | 일부 개선 가능 |

### 실패 원인 분석
| 원인 | 설명 |
|------|------|
| **시장 레짐 변화** | Train: 평균회귀 패턴 → Test: 일방적 상승 추세 |
| **Naive 강점** | 추세 지속 시장에서 Naive가 자연스럽게 우위 |
| **과적합** | 복잡한 모델이 과거 패턴에 과적합 |
| **소규모 Test** | 12주 샘플로 통계적 유의성 확보 어려움 |

### 교훈
1. **단순함의 가치** - 복잡한 모델 ≠ 좋은 모델
2. **시장 레짐 중요** - 모델 전에 시장 상태 파악 필수
3. **Naive 기반 Hybrid** - 금융 시계열의 강력한 베이스라인

### 향후 연구 방향
- Market Regime Detector 개발
- 동적 가중치 조절 시스템
- 더 긴 Test 기간으로 재검증

---
**End of Notebook**

